# RPA-based reverse-engineering

In [ ]:
from collections import Counter
import numpy as np
import holoviews as hv
from scipy.signal import find_peaks

from pyecsca.ec.key_generation import KeyGeneration
from pyecsca.ec.key_agreement import ECDH_SHA1
from pyecsca.ec.model import ShortWeierstrassModel
from pyecsca.ec.coordinates import AffineCoordinateModel
from pyecsca.ec.curve import EllipticCurve
from pyecsca.ec.params import DomainParameters
from pyecsca.ec.formula import FormulaAction
from pyecsca.ec.point import Point
from pyecsca.ec.mod import Mod
from pyecsca.ec.mult import *
from pyecsca.sca.trace.sampling import downsample_average, downsample_max
from pyecsca.sca.trace.process import normalize, rolling_mean
from pyecsca.sca.trace.combine import average, subtract
from pyecsca.sca.trace.test import welch_ttest
from pyecsca.sca.attack.leakage_model import HammingWeight, NormalNoice
from pyecsca.ec.context import DefaultContext, local
from pyecsca.sca.re.rpa import MultipleContext, rpa_distinguish
from pyecsca.sca.trace import Trace
from pyecsca.sca.trace.plot import plot_trace, plot_traces

In [ ]:
hv.extension("bokeh")

In [ ]:
model = ShortWeierstrassModel()
coordsaff = AffineCoordinateModel(model)
coords = model.coordinates["projective"]
add = coords.formulas["add-2007-bl"]  # The formulas are irrelevant for this method
dbl = coords.formulas["dbl-2007-bl"]
neg = coords.formulas["neg"]

# A 64-bit prime order curve for testing things out
p = 0xc50de883f0e7b167
a = Mod(0x4833d7aa73fa6694, p)
b = Mod(0xa6c44a61c5323f6a, p)
gx = Mod(0x5fd1f7d38d4f2333, p)
gy = Mod(0x21f43957d7e20ceb, p)
n = 0xc50de885003b80eb
h = 1

# A (0, y) RPA point on the above curve, in affine coords.
P0_aff = Point(coordsaff, x=Mod(0, p), y=Mod(0x1742befa24cd8a0d, p))

infty = Point(coords, X=Mod(0, p), Y=Mod(1, p), Z=Mod(0, p))
g = Point(coords, X=gx, Y=gy, Z=Mod(1, p))

curve = EllipticCurve(model, coords, p, infty, dict(a=a,b=b))
params = DomainParameters(curve, g, n, h)

## Exploration
First select a bunch of multipliers. We will be trying to distinguish among these.

In [ ]:
multipliers = []
multipliers.append(LTRMultiplier(add, dbl, None, False, True, True))
multipliers.append(LTRMultiplier(add, dbl, None, True, True, True))
multipliers.append(RTLMultiplier(add, dbl, None, False, True))
multipliers.append(RTLMultiplier(add, dbl, None, True, True))
multipliers.append(SimpleLadderMultiplier(add, dbl, None, True, True))
multipliers.append(BinaryNAFMultiplier(add, dbl, neg, None, True))
multipliers.append(WindowNAFMultiplier(add, dbl, neg, 3, None, True))
multipliers.append(WindowNAFMultiplier(add, dbl, neg, 4, None, True))

Then select a random scalar and simulate computation using all of the multipliers, track the multiples, print the projective and affine results.

In [ ]:
scalar = 0b1000000000000000000000000000000000000000000000000
scalar = 0b1111111111111111111111111111111111111111111111111
scalar = 0b1010101010101010101010101010101010101010101010101
scalar = 0b1111111111111111111111110000000000000000000000000
scalar = 123456789123456789
# multiples is a mapping from a multiple (integer) to a set of scalar multipliers that compute said multiple when doing [scalar]
multiples = {}

for mult in multipliers:
    print(repr(mult))
    with local(MultipleContext()) as ctx:
        mult.init(params, g)
        res = mult.multiply(scalar)
    print(res, res.to_affine())
    for m in ctx.points.values():
        s = multiples.setdefault(m, set())
        s.add(mult)
    print()


Pick a multiple `k` that is computed by some multiplier for the scalar,
invert it mod n, and do `[k^-1]P0` to obtain a point `P0_target`,
such that, `[k]P0_target = P0` and `P0` has a zero coordinate.

In [ ]:
k = 7186132
kinv = Mod(k, n).inverse()
P0_target = curve.affine_multiply(P0_aff, int(kinv)).to_model(coords, curve)

print("Original P0", P0_aff)
print("P0_target  ", P0_target.to_affine())
print("Verify P0  ", curve.affine_multiply(P0_target.to_affine(), k))

Now go over the multipliers with P0_target and the original scalar as input.
Then look whether a zero coordinate point was computed.
Also look at whether the multiple "k" was computed. These two should be the same.

In [ ]:
for mult in multipliers:
    print(mult.__class__.__name__)
    with local(MultipleContext()) as ctx:
        mult.init(params, P0_target)
        res = mult.multiply(scalar)
    print("\tzero present     ", any(map(lambda P: P.X == 0, ctx.points.keys())))
    print("\tmultiple computed", k in ctx.points.values())
    print()

Now lets look at the relation of multiples to multipliers.

In [ ]:
for multiple, mults in multiples.items():
    print(multiple, [mult.__class__.__name__ for mult in mults])

## Reverse-engineering
The `simulated_oracle` function simulates an RPA oracle that detect a zero coordinate point in the scalar multiplication.
This can be used by the `rpa_distinguish` function to distinguish the true scalar multiplier.

In [ ]:
def simulated_oracle(scalar, affine_point):
    real_mult = BinaryNAFMultiplier(add, dbl, neg, None, True)
    point = affine_point.to_model(params.curve.coordinate_model, params.curve)
    with local(MultipleContext()) as ctx:
        real_mult.init(params, point)
        real_mult.multiply(scalar)
    return any(map(lambda P: P.X == 0 or P.Y == 0, ctx.points.keys()))

In [ ]:
rpa_distinguish(params, multipliers, simulated_oracle)

The `simulate_trace` function simulates a Hamming weight leakage trace of a given multiplier computing a scalar multiple.
This is used by the `simulated_rpa_trace` function that does the RPA attack on simulated traces and returns the differential
trace. This is in turn used to build the `simulated_rpa_oracle` which can be used by the `rpa_distinguish` function to perform
RPA-RE and distinguish the true scalar multiplier.

In [ ]:
def simulate_trace(mult, scalar, point):
    with local(DefaultContext()) as ctx:
        mult.init(params, point)
        mult.multiply(scalar)

    lm = HammingWeight()
    trace = []

    def callback(action):
        if isinstance(action, FormulaAction):
            for intermediate in action.op_results:
                leak = lm(intermediate.value)
                trace.append(leak)

    ctx.actions.walk(callback)
    return Trace(np.array(trace))

def simulated_rpa_trace(mult, scalar, affine_point, noise):
    target_point = affine_point.to_model(params.curve.coordinate_model, params.curve)
    random_point = params.curve.affine_random().to_model(params.curve.coordinate_model, params.curve)

    random_traces = [noise(simulate_trace(mult, scalar, random_point)) for _ in range(10)]
    target_traces = [noise(simulate_trace(mult, scalar, target_point)) for _ in range(500)]

    random_avg = average(*random_traces)
    target_avg = average(*target_traces)

    diff_trace = downsample_max(subtract(random_avg, target_avg), 25)
    return diff_trace

def simulated_rpa_oracle(scalar, affine_point):
    real_mult = BinaryNAFMultiplier(add, dbl, neg, None, True)
    noise = NormalNoice(0, 1)
    diff_trace = normalize(simulated_rpa_trace(real_mult, scalar, affine_point, noise))
    peaks, props = find_peaks(diff_trace.samples, height=4)
    return len(peaks) != 0

In [ ]:
rpa_distinguish(params, multipliers, simulated_rpa_oracle)

Note that the oracle function above has several parameters, like noise amplitude, amount of traces simulated and peak finding height threshold. The cell below compares the differential RPA trace when the multiple is computed in the simulation vs when it is not.

In [ ]:
diff_real = normalize(simulated_rpa_trace(BinaryNAFMultiplier(add, dbl, neg, None, True), scalar, P0_target.to_affine(), NormalNoice(0, 1)))
diff_nothing = normalize(simulated_rpa_trace(LTRMultiplier(add, dbl, None, False, True, True), scalar, P0_target.to_affine(), NormalNoice(0, 1)))
plot_traces(diff_real, diff_nothing).opts(width=950, height=600)